In [ ]:
import os
import pandas as pd

: 

In [ ]:
"""Determine file path"""
path = os.path.dirname(os.path.abspath("eapps_processing.ipynb"))
print(path)

: 

In [ ]:
"""Create dataframes from new and previous results"""

raw_path = fr"{path}\raw\results.csv"
processed_path = fr"{path}\processed\results.csv"

df = pd.read_csv(raw_path)
processed_df = pd.read_csv(processed_path)

: 

In [ ]:
df = df.dropna(subset=["case_status", "case_number", "architect", "applicant", "description"])

: 

In [ ]:
# Remove "Dr" and "Perit" and clean up whitespace
df['architect'] = df['architect'].str.replace(r'\s*Dr\s*|\s*Perit\s*', '', regex=True).str.strip()

: 

In [ ]:
def reverse_names(name):
    exemptions = ["X,Y,Z Architecture & Design", "Mangion, Mangion & Partners", "Nois currently responsible for this case. X,Y,Z Architecture & Design relinquished responsibility on 20 July 2020", "Innovative Design Architects (iDA) 23, Triq Titu B"]
    if name not in exemptions:
        parts = name.split(',')
        if len(parts) == 2:
            return f'{parts[1].strip()} {parts[0].strip()}'
        else:
            return name
    
df['architect'] = df['architect'].apply(reverse_names)

: 

In [ ]:
df['applicant'] = df['applicant'].str.replace('.*Attn: ', '', regex=True)
df['applicant'] = df['applicant'].str.replace('.*Attn. ', '', regex=True)
df['applicant'] = df['applicant'].str.replace('.*Attn ', '', regex=True)
df['applicant'] = df['applicant'].str.replace('.*represented by ', '', regex=True)
df['applicant'] = df['applicant'].str.replace(' obo .*', '', regex=True)
df['applicant'] = df['applicant'].str.replace(' o.b.o .*', '', regex=True)
df['applicant'] = df['applicant'].str.replace(' o.b.o. .*', '', regex=True)
df['applicant'] = df['applicant'].str.replace(' o.b.o. .*', '', regex=True)
df['applicant'] = df['applicant'].str.replace(r'\s*\(o\.b\.o.*', '', regex=True)

: 

In [ ]:
df['applicant'] = df['applicant'].str.replace(r'^\s*(?:Mr\.|Mrs\.|Mr |Ms\.|Ms |Dr\.|Dr |Ing\.|Ing |Fr\.|Fr|Rev\.|Rev\. )\s*', '', regex=True)

: 

In [ ]:
df['applicant'] = df['applicant'].apply(lambda x: x.title() if isinstance(x, str) else x)

: 

In [ ]:
df[['application_type', 'application_number', 'application_year']] = df['case_number'].str.split('/', expand=True)

: 

In [ ]:
columns_to_process = ['description', 'applicant', 'architect', 'location']
df[columns_to_process] = df[columns_to_process].applymap(lambda x: x.replace("â€™", "'") if isinstance(x, str) else x)
df[columns_to_process] = df[columns_to_process].applymap(lambda x: x.replace('\u2019', "'") if isinstance(x, str) else x)

: 

In [ ]:
# Concatenate the dataframes
concatenated_df = pd.concat([processed_df, df], ignore_index=True)

# Keep only unique rows based on the 'Name' column
unique_df = concatenated_df.drop_duplicates(subset='case_number', keep='last')

# Display the updated dataframe
print(unique_df)

: 

In [ ]:
unique_df.to_csv(processed_path, encoding='utf-8', index=False)

: 